In [1]:
import pandas as pd
import glob
import pybedtools
from pybedtools import BedTool
import pathlib
import subprocess 

In [6]:
! bedtools makewindows -g mm10.txt -w 5000 > mm10_5000.bed

In [8]:
mm10_window.head()

,0,1,2,3,4
0,chr1,0,5000,a0,0
1,chr1,5000,10000,a1,1
2,chr1,10000,15000,a2,2
3,chr1,15000,20000,a3,3
4,chr1,20000,25000,a4,4


## gen map command

In [2]:
dmr_dir = "/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/mC_FILTER_DMR_BED"

In [14]:
for _type in ['Hypo','Hyper']:
    all_paths = glob.glob(f"{dmr_dir}/*.{_type}.aDMR.bed")
    for path in all_paths:
        ct = path.split('/')[-1].split('.')[0]
        tmpdf = pd.read_csv(path, sep = '\t', header = None)
        tmpdf['length'] = tmpdf[2] - tmpdf[1]
        tmpdf = tmpdf[[0,1,2,'length']]
        tmpdf.to_csv(f'DMR_BED/{ct}.{_type}.bed', sep = '\t', index = None, header = None)

In [19]:
tmpdf.head()

,0,1,2,length
0,chr1,6389498,6389512,14
1,chr1,9891886,9891900,14
2,chr1,14110488,14110536,48
3,chr1,14305202,14305205,3
4,chr1,14508392,14508419,27


In [30]:
pathlib.Path('5k_hotspot').mkdir(exist_ok = True)
for _type in ['Hypo','Hyper']:
    for file in glob.glob(f"DMR_BED/*.{_type}.bed"):
        mt = file.split('/')[-1].split('.')[0]
        command = f'bedtools map -a  mm10_5000.bed -b {file} -null 0 -c 4 > 5k_hotspot/{mt}.{_type}.map.bed'
        subprocess.run(command, shell =True)

## group command

In [2]:
for _type in ['Hypo','Hyper']:
    all_hypo_files = glob.glob(f'5k_hotspot/*.{_type}.map.bed')
    command = f'cat {" ".join(all_hypo_files)} > {_type}.concat.bed'
    subprocess.run(command, shell = True)
    subprocess.run(f"sort -k1,1 -k2,2n -k3,3n -o {_type}.concat.bed {_type}.concat.bed", shell = True)

    command = f"bedtools groupby -i {_type}.concat.bed -g 1,2,3 -c 6 -o sum > {_type}.concat.sum.bed"
    subprocess.run(command, shell = True)
    subprocess.run(f"rm {_type}.concat.bed", shell = True)